In [ ]:
%matplotlib inline

In [ ]:
import xarray as xr
import numpy as np
import torch
from lib.models.torch import interface
from lib.advection import vertical_advection

In [ ]:
def _compute_residual(x, f):
    dt = x.time[1]-x.time[0]
    return (f-f.shift(time=-1))/dt - f


def compute_residuals(inputs, forcings):
    """Compute Q1 and Q2"""
    ds = xr.Dataset({key: _compute_residual(inputs[key], forcings[key])
                for key in ['qt', 'sl']})
    
    return ds




def compare_srcs(src, resid, **kw):
    """Compare the x and time averaged source terms"""
    
    kw.update(dict(x="y", y="z"))
    fig, axs = plt.subplots(2, 2, figsize=(8, 6), sharey=True, sharex=True)
    src.sl.mean(['x', 'time']).plot(ax=axs[0,0], **kw)
    resid.sl.mean(['x', 'time']).plot(ax=axs[0,1], **kw)


    src.qt.mean(['x', 'time']).plot(ax=axs[1,0], **kw)
    resid.qt.mean(['x', 'time']).plot(ax=axs[1,1], **kw)

    for ax, title in zip(axs.flat,["NN Q1", "Q1", "NN Q2", "Q2"]):
        ax.set_title(title)

In [ ]:
inputs = xr.open_dataset("../data/processed/inputs.nc")
forcings = xr.open_dataset("../data/processed/forcings.nc")

resid = compute_residuals(inputs, forcings)

In [ ]:
model = torch.load("../data/output/model.test/1.torch")
src = interface.rhs(model, inputs, forcings)

In [ ]:
compare_srcs(src, resid)

There are some very big differences between the prediction and the observed residuals. I bet this is because I perform this verification without first adding the forcing term. The actual time stepping taken by the neural network looks like
$$ x^* = x^n + \frac{h}{2}(g^{n+1} + g^n),\quad x^{n+1} = x^* + h f(x^*) $$
where $g^n$ is the advection forcing at time step $n$. There are probably very big splitting errors because I am using $h=3$ hours, which means that the statistics of $x^*$ might be very different from the statistics of $x^n$. This could have a large impact on the output of the neural network. To do this let's first apply the advection terms to the input data.

In [ ]:
def trapezoid_step(x, g, h=.125):
    return x + h * (g + g.shift(time=1))/2

xst = inputs.apply(lambda x: trapezoid_step(x, forcings[x.name]) if x.name in ['sl', 'qt'] else x)
gavg = (forcings + forcings.shift(time=1))/2

Now let's compare the mean of $x^*$ and $x^n$.

In [ ]:
bias = xst.mean(['x', 'time']) - inputs.mean(['x', 'time'])

In [ ]:
fig, (aq, at)  = plt.subplots(1, 2, figsize=(8,3), sharey=True)

bias.qt.plot(ax=aq)
bias.sl.plot(ax=at)

aq.set_title("QT diff (g/kg)")
aq.set_title("SL diff (K)")

These differences are actually pretty small, so I would be surprised if they are causing the problem, but let's just see what the Neural network predicts the source terms will be.

In [ ]:
src = interface.rhs(model, xst, gavg)

In [ ]:
compare_srcs(src, resid)

This made a big difference! It turns out the splitting error actually matters a lot in this scenario.

# Analyzing Error

In [ ]:
err = xr.open_dataset("../data/output/test_error.nc")
err= err.assign(qt64=(err.qt*err.w).mean('z'), sl64=(err.sl*err.w).mean('z'))

In [ ]:
def bootstrap(stat, data, n=1000):
    stats = []
    for i in range(n):
        sample = np.random.choice(data, data.shape[0])
        stats.append(stat(sample))
    
    return stats

def get_ci(x, interval=(2.5, 97.5)):
    samples = bootstrap(np.median, x)
    a, b = np.percentile(samples, interval)
    return np.median(x), (b-a)/2


def get_cis(x):
    return x.apply(get_ci, axis=0)


def format_uncertain(x):
    if x[0] > 1000:
        return r'$\infty$'
    else:
        return f"{x[0]:.2f} ({x[1]:.2f})"

In [ ]:
df = err[['qtR2', 'slR2', 'qt64', 'sl64', 'nhidden', 'window_size']]\
.to_dataframe()


df_with_ci = df.groupby(['nhidden', 'window_size'])\
.apply(get_cis)\
.drop('window_size', 1)\
.drop('nhidden', 1)\
.applymap(format_uncertain)

In [ ]:
columns = [
    ('qtR2',  (r'Apparent Source $R^2$', r'$q_T$')),
    ('slR2',  (r'Apparent Source $R^2$', r'$s_L$')),
    ('qt64',  (r'64 Step Error', r'$q_T$')),
    ('sl64',  (r'64 Step Error', r'$s_L$')), 
]

cols = pd.MultiIndex.from_tuples(dict(columns).values())
df_with_ci.columns = cols

In [ ]:
df_with_ci

In [ ]:
vary_nhid = df_with_ci.loc[pd.IndexSlice[:, 10],:].reset_index()\
.drop('window_size', 1)\
.rename({"nhidden": r"$n$"}, axis=1)

vary_t = df_with_ci.loc[pd.IndexSlice[128,:],:].reset_index()\
.drop('nhidden', 1)\
.rename({"window_size": r"$T$"}, axis=1)

In [ ]:
vary_t

In [ ]:
vary_nhid